In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive/hw2/lstm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from gensim.models import Word2Vec
from sklearn.metrics import classification_report

In [ ]:
INPUT_FOLDER = '.'
OUTPUT_FOLDER = 'csv_data'

In [ ]:
import pandas as pd

def load_yelp_orig_data():
    PATH_TO_YELP_REVIEWS = INPUT_FOLDER + '/review.json'

    # read the entire file into a python array
    with open(PATH_TO_YELP_REVIEWS, 'r') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    data_json_str = "[" + ','.join(data) + "]"

    # now, load it into pandas
    data_df = pd.read_json(data_json_str)

    data_df.head(100000).to_csv(OUTPUT_FOLDER + '/output_reviews_top.csv')

load_yelp_orig_data()

In [ ]:
top_data_df = pd.read_csv(INPUT_FOLDER + '/csv_data/output_reviews_top.csv')
print("Columns in the original dataset:\n")
print(top_data_df.columns)

In [ ]:
print("Number of rows per star rating:")
print(top_data_df['stars'].value_counts())

In [ ]:
# Function to map stars to sentiment
def map_sentiment(stars_received):
    if stars_received <= 2:
        return -1
    elif stars_received == 3:
        return 0
    else:
        return 1

# Mapping stars to sentiment into three categories
top_data_df['sentiment'] = [ map_sentiment(x) for x in top_data_df['stars']]

# Plotting the sentiment distribution
plt.figure()
pd.value_counts(top_data_df['sentiment']).plot.bar(title="Sentiment distribution in df")
plt.xlabel("Sentiment")
plt.ylabel("No. of rows in df")
plt.show()

In [ ]:
# Function to retrieve top few number of each category
def get_top_data(top_n = 5000):
    top_data_df_positive = top_data_df[top_data_df['sentiment'] == 1].head(top_n)
    top_data_df_negative = top_data_df[top_data_df['sentiment'] == -1].head(top_n)
    top_data_df_neutral = top_data_df[top_data_df['sentiment'] == 0].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive, top_data_df_negative, top_data_df_neutral])
    return top_data_df_small

# Function call to get the top 10000 from each sentiment
top_data_df_small = get_top_data(top_n=10000)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(top_data_df_small['sentiment'].value_counts())
top_data_df_small.head(10)

In [ ]:
# Tokenize the text column to get the new column 'tokenized_text'
top_data_df_small['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in top_data_df_small['text']] 
print(top_data_df_small['tokenized_text'].head(10))

In [ ]:
porter_stemmer = PorterStemmer()
# Get the stemmed_tokens
top_data_df_small['stemmed_tokens'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in top_data_df_small['tokenized_text'] ]
top_data_df_small['stemmed_tokens'].head(10)

In [ ]:
top_data_df_small['funny'] = [eval(x)['funny'] for x in top_data_df_small['votes']]
top_data_df_small['cool'] = [eval(x)['cool'] for x in top_data_df_small['votes']]
top_data_df_small['useful'] = [eval(x)['useful'] for x in top_data_df_small['votes']]

In [ ]:
# Train Test Split Function
def split_train_test(top_data_df_small, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(top_data_df_small[['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text', 'useful', 'user_id', 'stemmed_tokens']], 
                                                        top_data_df_small['sentiment'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

In [ ]:
# Use cuda if present
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

In [ ]:
def make_word2vec_model(top_data_df_small, INPUT_FOLDER, padding=True, sg=1, min_count=1, size=500, workers=3, window=3):
    if  padding:
        print(len(top_data_df_small))
        temp_df = pd.Series(top_data_df_small['stemmed_tokens']).values
        temp_df = list(temp_df)
        temp_df.append(['pad'])
        word2vec_file = INPUT_FOLDER + '/models/' + 'word2vec_' + str(size) + '_PAD.model'
    else:
        temp_df = top_data_df_small['stemmed_tokens']
        word2vec_file = INPUT_FOLDER + '/models/' + 'word2vec_' + str(size) + '.model'
    w2v_model = Word2Vec(temp_df, min_count = min_count, vector_size=size, workers = workers, window = window, sg = sg)
    
    w2v_model.save(word2vec_file)
    return w2v_model, word2vec_file

In [ ]:
from numpy import vectorize
size = 500
window = 3
min_count = 1
workers = 3
sg = 1

# train Word2vec model
w2vmodel, word2vec_file = make_word2vec_model(top_data_df_small, INPUT_FOLDER, padding=True, sg=sg, min_count=min_count, size=size, workers=workers, window=window)
max_sen_len = top_data_df_small.stemmed_tokens.map(len).max()
padding_idx = w2vmodel.wv.index_to_key.index('pad')

In [ ]:
def make_word2vec_vector_lstm(sentence, w2vmodel, max_sen_len, padding_idx, device):
    padded_X = [padding_idx for i in range(max_sen_len)]
    i = 0
    for word in sentence:
        if word not in w2vmodel.wv.index_to_key:
            padded_X[i] = 0
            print(word)
        else:
            padded_X[i] = w2vmodel.wv.index_to_key.index(word)
        i += 1
    return torch.tensor(padded_X, dtype=torch.long, device=device).view(1, -1)

In [ ]:
def make_target(label, device):
    if label == -1:
        return torch.tensor([0], dtype=torch.long, device=device)
    elif label == 0:
        return torch.tensor([1], dtype=torch.long, device=device)
    else:
        return torch.tensor([2], dtype=torch.long, device=device)

In [ ]:
EMBEDDING_SIZE = 500
HIDDEN_SIZE= 500
NUM_FILTERS = 10
import gensim

class LstmTextClassifier(nn.Module):
    def __init__(self, vocab_size, num_classes, INPUT_FOLDER, HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS=1):
        super(LstmTextClassifier, self).__init__()
        w2vmodel = gensim.models.KeyedVectors.load(INPUT_FOLDER + '/models/' + 'word2vec_500_PAD.model')
        weights = w2vmodel.wv
        # With pretrained embeddings
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights.vectors), padding_idx=w2vmodel.wv.index_to_key.index('pad'))
        
        self.lstm = nn.LSTM(input_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, batch_first=True)

        self.fc = nn.Linear(HIDDEN_SIZE, num_classes)

    def forward(self, x):
        x = self.embedding(x) # [B, T, E]
        out, _ = self.lstm(x) # [B, T, H]
        
        # Extract the last hidden state of the sequence as the representation for classification
        last_hidden = out[:, -1, :]
        logits = self.fc(last_hidden)
        probs = F.softmax(logits, dim=1)

        return probs


In [ ]:
NUM_CLASSES = 3
VOCAB_SIZE = len(w2vmodel.wv.index_to_key)

lstm_model = LstmTextClassifier(vocab_size=VOCAB_SIZE, num_classes=NUM_CLASSES, INPUT_FOLDER=INPUT_FOLDER, HIDDEN_SIZE=HIDDEN_SIZE, EMBEDDING_SIZE=EMBEDDING_SIZE)

lstm_model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
num_epochs = 30

# Open the file for writing loss
loss_file_name = INPUT_FOLDER +  '/plots/' + 'lstm_class_big_loss_with_padding.csv'
f = open(loss_file_name,'w')
f.write('iter, loss')
f.write('\n')
losses = []
lstm_model.train()

for epoch in range(num_epochs):
    print("Epoch " + str(epoch + 1))
    train_loss = 0
    for index, row in X_train.iterrows():
        # Clearing the accumulated gradients
        lstm_model.zero_grad()

        # Make the bag of words vector for stemmed tokens 
        bow_vec = make_word2vec_vector_lstm(row['stemmed_tokens'], w2vmodel, max_sen_len, padding_idx, device)
       
        # Forward pass to get output
        probs = lstm_model(bow_vec)

        # Get the target label
        target = make_target(Y_train['sentiment'][index], device)

        # Calculate Loss: softmax --> cross entropy loss
        loss = loss_function(probs, target)
        train_loss += loss.item()

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

    print("Epoch ran :"+ str(epoch+1))
    f.write(str((epoch+1)) + "," + str(train_loss / len(X_train)))
    f.write('\n')
    train_loss = 0

torch.save(lstm_model, INPUT_FOLDER + 'lstm_big_model_500_with_padding.pth')
f.close()

In [ ]:
print("Input vector")
print(bow_vec.cpu().numpy())
print("Probs")
print(probs)
print(torch.argmax(probs, dim=1).cpu().numpy()[0])

In [ ]:
bow_lstm_predictions = []
original_lables_lstm_bow = []
lstm_model.eval()
loss_df = pd.read_csv(INPUT_FOLDER + '/plots/' + 'lstm_class_big_loss_with_padding.csv')
print(loss_df.columns)
with torch.no_grad():
    for index, row in X_test.iterrows():
        bow_vec = make_word2vec_vector_lstm(row['stemmed_tokens'], w2vmodel, max_sen_len, padding_idx, device)
        probs = lstm_model(bow_vec)
        _, predicted = torch.max(probs.data, 1)
        bow_lstm_predictions.append(predicted.cpu().numpy()[0])
        original_lables_lstm_bow.append(make_target(Y_test['sentiment'][index], device).cpu().numpy()[0])
print(classification_report(original_lables_lstm_bow,bow_lstm_predictions))
loss_file_name = INPUT_FOLDER +  '/plots/' + 'lstm_class_big_loss_with_padding.csv'
loss_df = pd.read_csv(loss_file_name)
print(loss_df.columns)
plt_500_padding_30_epochs = loss_df[' loss'].plot()
fig = plt_500_padding_30_epochs.get_figure()
fig.savefig(INPUT_FOLDER +'/plots/' + 'loss_plt_500_padding_30_epochs.pdf')